In [447]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime,date
import re
import json
from pandas.api.types import is_numeric_dtype

# from simple_salesforce import Salesforce, format_soql, format_external_id
# from simple_salesforce.exceptions import SalesforceMalformedRequest
# import simpler_sf
from sqlalchemy import create_engine
#import pymysql

# from mysql import connector


# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [448]:
# usuario = 'siempreread'
# contrasena = 'S13mpr3_Ch1l3'
# url_servidor = '209.151.148.250'
puerto = "3306"
esquema = "MetLife_A"
plugin_autenticacion = "mysql_native_password"

usuario = "care_consulta"
contrasena = "CareAssistance!10"
url_servidor = "10.0.10.22"

# Conexión a la base de datos

c_conexion = "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}"
c_conexion = c_conexion.format(
    usuario, contrasena, url_servidor, puerto, esquema, plugin_autenticacion
)
motor_mysql_mariadb = create_engine(c_conexion)

#table.to_sql(name='rut_unico_tiempo',con=motor_mysql_mariadb,if_exists = 'append')


# archivo de salida modificar segun archivo
# archivo = 'C:/Users/jmartinez/Desktop/casos nuevos/clientes/1/casos_faltantes_segun_archivo.xlsx'

In [449]:
# table.to_sql(name='prueba',con=motor_mysql_mariadb,if_exists = 'append')
rut_unicos = pd.DataFrame()
sql = "SELECT * FROM MetLife_A.rut_unicos"
rut_unicos = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("-", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("–", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(".", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(" ", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.upper()
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.strip()

rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("-", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("–", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(".", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(" ", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.upper()
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.strip()

rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(",", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(".", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("-", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┐", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("═", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("╔", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┌", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.title()
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.strip()

rut_unicos.shape

In [ ]:
rut_unico_tiempo_ = pd.DataFrame()
sql = "SELECT Llave FROM MetLife_A.rut_unico_tiempo"
rut_unico_tiempo_ = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unico_tiempo_.head()

,Llave
0,34002302717805539912023
1,34000243618617790812023
2,34002273710579623412023
3,34002317316849726112023
4,34000428510812004512023


In [ ]:
rut_unico_tiempo_.shape

(2343498, 1)

In [ ]:
def fechaValida(fecha):
    if (len(str(fecha)) <= 9 ):
        fecha = ""
        
    fecha = str(fecha)
    fecha1 = str(fecha)
    
    #datetime.strptime(fecha, '%Y-%m-%d')
    
    try:
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        date_string = fecha1
        #print("date_string =", date_string)
        #print("type of date_string =", type(date_string))

        date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

        #print("date_object =", date_object)
        #print("type of date_object =", type(date_object))
        return date_object
    except ValueError:
        return datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
mes = 8
año = "2023"
fecha = año + "-" + str(mes) + "-" + "01"  
mesesDic = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

str(mesesDic[mes])



sftp = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/sftp.xlsx"


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"


derty = pd.read_excel(sftp)

In [ ]:
derty.shape

(431771, 20)

In [ ]:
metlife_sftp = pd.DataFrame()
metlife_sftp = derty.copy()

In [ ]:


metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("-", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("–", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(" ", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.upper()
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.strip()

metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("-", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("–", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(" ", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.upper()
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.strip()

metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(",", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(".", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("-", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┐", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("═", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("╔", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┌", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.title()
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.strip()

metlife_sftp["poliza"] = metlife_sftp["poliza"].astype(str)
metlife_sftp["poliza"] = metlife_sftp["poliza"].str.strip()



#*******************sacar primer cero
metlife_sftp.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\3662042864.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\3662042864.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\3662042864.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["Nombre"] = metlife_sftp["Nomb

(431771, 20)

In [ ]:
metlife_correo.head()

,poliza,GRUPO,NOMBREGRUPO,RUTASEGURADO,DV,rut2,CARGA,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,...,NOMBREBANCO,NUMEROCUENTACORRIENTE,CAPITALASEGFALLECIMIENTO,CAPITALASEGMUERTEACC_AP,NROCELULAR,IDBENEFICIARIO,POLGR,NUMEROASEGURADO,NUMEROCARGA,Titular
0,340020794,1.0,Rol Ejecutivo UF 4000,10173226,6,101732266,0.0,10173226,6,101732266,...,BANCO DE CREDITO E INVERSIONES,3.655407e+07,4000.0,0.0,995427700,75755945.0,0340020794G0001,1.0,0.0,1
3,340020794,4.0,Rol Empleados UF 750,5818805,0,58188050,0.0,5818805,0,58188050,...,BANCO DE CREDITO E INVERSIONES,5.892857e+07,750.0,0.0,989112063,75755948.0,0340020794G0004,1.0,0.0,1
4,340020794,4.0,Rol Empleados UF 750,6369895,4,63698954,0.0,6369895,4,63698954,...,BANCO SANTANDER-CHILE,2.375575e+06,750.0,0.0,NaN,75755951.0,0340020794G0004,4.0,0.0,1
6,340020794,4.0,Rol Empleados UF 750,6512512,9,65125129,0.0,6512512,9,65125129,...,BANCO DE CREDITO E INVERSIONES,1.394650e+07,750.0,0.0,971934727,80602175.0,0340020794G0004,1630.0,0.0,1
7,340020794,4.0,Rol Empleados UF 750,6534977,9,65349779,0.0,6534977,9,65349779,...,Banco de Chile,1.660293e+09,750.0,0.0,NaN,77777959.0,0340020794G0004,1401.0,0.0,1


In [ ]:
def siPrimerCero(y):    
    if len(y) <= 4:
        y =  "NONE"
    if y[0] == "0":
        y = y[1:]
    return str(y)
    

In [ ]:
metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_sftp = metlife_sftp[(metlife_sftp["rut1"] != "NONE")]
metlife_sftp = metlife_sftp[(metlife_sftp["rut2"] != "NONE")]


metlife_sftp["Titular"] = np.where(metlife_sftp["rut1"] == metlife_sftp["rut2"],1,2 )
metlife_sftp = metlife_sftp[(metlife_sftp["Titular"] == 1)]
metlife_sftp = metlife_sftp.drop_duplicates(
    [
        "rut1","poliza",
    ],
    keep="last",
)

metlife_sftp.shape

(402734, 21)

In [ ]:
correo = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/correo.xlsx"
dertyc = pd.read_excel(correo)
metlife_correo = dertyc.copy()

# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("-", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("–", "")
metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(" ", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.upper()
metlife_correo["rut1"] = metlife_correo["rut1"].str.strip()

metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("-", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("–", "")
metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(" ", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.upper()
metlife_correo["rut2"] = metlife_correo["rut2"].str.strip()

metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(",", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(".", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("-", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┐", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("═", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("╔", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┌", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.title()
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.strip()

metlife_correo["poliza"] = metlife_correo["poliza"].astype(str)
metlife_correo["poliza"] = metlife_correo["poliza"].str.strip()




#*******************sacar primer cero
metlife_correo.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\2005110109.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\2005110109.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\2005110109.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["Nombre"] = metlife

(490, 45)

In [ ]:
metlife_correo["rut1"] = metlife_correo["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut2"] = metlife_correo["rut2"].apply(lambda y: siPrimerCero(y))
metlife_correo = metlife_correo[(metlife_correo["rut1"] != "NONE")]
metlife_correo = metlife_correo[(metlife_correo["rut2"] != "NONE")]

metlife_correo["Titular"] = np.where(metlife_correo["rut1"] == metlife_correo["rut2"],1,2 )
metlife_correo = metlife_correo[(metlife_correo["Titular"] == 1)]
metlife_correo = metlife_correo.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)

metlife_correo.shape

(269, 46)

In [ ]:
web = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/web.xlsx"
dertyd = pd.read_excel(web)


In [ ]:
dertyd.head(9)

,id,ciudad,codigo_carga,codigo_postal,comuna,convenio,convenio_plan,direccion,fecha_bloqueo,fecha_modificacion,...,Nombre,poliza,p_relacion,rut1,rut2,dsfsdfsd,p_sexo,procesado,rut_empresa,version
0,233835,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-07-01,...,GREGORIO VALENZUELA BUSTOS,340002436,1,163993139,163993139,1,1,0,898622002,1
1,233836,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-07-01,...,LUISANA DEL VAL TOMEDES,340002446,1,254035122,254035122,1,2,0,969512807,1
2,233837,NaN,0,NaN,NaN,77,SIN PLAN,RETIRA,0,2023-07-01,...,ROBERTO SANHUEZA HERNANDEZ,330000004,1,53718167,53718167,1,1,0,650086147,1
3,233843,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-07-01,...,VALERIA MUNOZ CAMUS,340000577,1,165002008,165002008,1,2,0,965404902,1
4,233845,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-07-01,...,JUAN ESTEBAN PEÑA PALMA,340001759,1,168184794,168184794,1,1,0,761574302,1
5,233848,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-07-01,...,JOSE MANUEL LAGOS CORNEJO,340001759,1,144967186,144967186,1,1,0,761574302,1
6,233849,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-07-01,...,MAURICIO ULLOA AMESTICA,340001376,1,13957611K,13957611K,1,1,0,700347001,1
7,233850,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-07-01,...,JUAN CELIS CABEZAS,340001376,1,132544328,132544328,1,1,0,700347001,1
8,233855,NaN,0,NaN,NaN,80,SIN PLAN,NaN,0,2023-07-01,...,ALEXIS MARCELIN ACUNA LEDEZMA,340010027,1,158018586,158018586,1,1,0,919150009,1


In [ ]:
metlife_web = dertyd.copy()

In [ ]:
metlife_web.head()

,id,ciudad,codigo_carga,codigo_postal,comuna,convenio,convenio_plan,direccion,fecha_bloqueo,fecha_modificacion,...,Nombre,poliza,p_relacion,rut1,rut2,dsfsdfsd,p_sexo,procesado,rut_empresa,version
0,233835,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-07-01,...,GREGORIO VALENZUELA BUSTOS,340002436,1,163993139,163993139,1,1,0,898622002,1
1,233836,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-07-01,...,LUISANA DEL VAL TOMEDES,340002446,1,254035122,254035122,1,2,0,969512807,1
2,233837,NaN,0,NaN,NaN,77,SIN PLAN,RETIRA,0,2023-07-01,...,ROBERTO SANHUEZA HERNANDEZ,330000004,1,53718167,53718167,1,1,0,650086147,1
3,233843,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-07-01,...,VALERIA MUNOZ CAMUS,340000577,1,165002008,165002008,1,2,0,965404902,1
4,233845,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2023-07-01,...,JUAN ESTEBAN PEÑA PALMA,340001759,1,168184794,168184794,1,1,0,761574302,1


In [ ]:


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_web["rut1"] = metlife_web["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut1"] = metlife_web["rut1"].str.replace("-", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace("–", "")
metlife_web["rut1"] = metlife_web["rut1"].astype(str)
metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace(" ", "")
metlife_web["rut1"] = metlife_web["rut1"].str.upper()
metlife_web["rut1"] = metlife_web["rut1"].str.strip()

metlife_web["rut2"] = metlife_web["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_web["rut2"] = metlife_web["rut2"].str.replace("-", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace("–", "")
metlife_web["rut2"] = metlife_web["rut2"].astype(str)
metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace(" ", "")
metlife_web["rut2"] = metlife_web["rut2"].str.upper()
metlife_web["rut2"] = metlife_web["rut2"].str.strip()

metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(",", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(".", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("-", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┐", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("═", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("╔", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┌", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.title()
metlife_web["Nombre"] = metlife_web["Nombre"].str.strip()

metlife_web["poliza"] = metlife_web["poliza"].astype(str)
metlife_web["poliza"] = metlife_web["poliza"].str.strip()

metlife_web["rut1"] = metlife_web["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut2"] = metlife_web["rut2"].apply(lambda y: siPrimerCero(y))
metlife_web = metlife_web[(metlife_web["rut1"] != "NONE")]
metlife_web = metlife_web[(metlife_web["rut2"] != "NONE")]
metlife_web.shape

metlife_web["Titular"] = np.where(metlife_web["rut1"] == metlife_web["rut2"],1,2 )
metlife_web = metlife_web[(metlife_web["Titular"] == 1)]

metlife_web = metlife_web.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)


#*******************sacar primer cero
metlife_web.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\2932449918.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\2932449918.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\2932449918.py:26: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["Nombre"] = metlife_web["Nombre"].

(6175, 42)

In [ ]:
subir_correo = pd.DataFrame()
subir_correo["poliza_rut"] = metlife_correo["poliza"]
subir_correo["rut_rut"] = metlife_correo["rut1"]
subir_correo["nombre_rut"] = metlife_correo["Nombre"]
subir_correo["mes_rut"] = str(mes)
subir_correo["año_rut"] = str(año)
#subir_correo["fecha_rut"] = metlife_correo["fecha"]
subir_correo["Llave"] = metlife_correo["poliza"] + metlife_correo["rut1"] + str(mes) + str(año)
subir_correo["desde_rut"] = metlife_correo["desde"].apply(lambda y: fechaValida(y))
subir_correo["hasta_rut"] = metlife_correo["hasta"].apply(lambda y: fechaValida(y))

subir_sql = pd.DataFrame()
subir_sql["poliza_rut"] = metlife_sftp["poliza"]
subir_sql["rut_rut"] = metlife_sftp["rut1"]
subir_sql["nombre_rut"] = metlife_sftp["Nombre"]
subir_sql["mes_rut"] = str(mes)
subir_sql["año_rut"] = str(año)
#subir_sql["fecha_rut"] = metlife_sftp["fecha"]
subir_sql["Llave"] = metlife_sftp["poliza"] + metlife_sftp["rut1"] + str(mes) + str(año)
subir_sql["desde_rut"] = metlife_sftp["desde"].apply(lambda y: fechaValida(y))
subir_sql["hasta_rut"] = metlife_sftp["hasta"].apply(lambda y: fechaValida(y)) 

subir_web = pd.DataFrame()
subir_web["poliza_rut"] = metlife_web["poliza"]
subir_web["rut_rut"] = metlife_web["rut1"]
subir_web["nombre_rut"] = metlife_web["Nombre"]
subir_web["mes_rut"] = str(mes)
subir_web["año_rut"] = str(año)
#subir_web["fecha_rut"] = metlife_web["fecha"]
subir_web["Llave"] = metlife_web["poliza"] + metlife_web["rut1"] + str(mes) + str(año)
subir_web["desde_rut"] = metlife_web["desde"].apply(lambda y: fechaValida(y))
subir_web["hasta_rut"] = metlife_web["hasta"].apply(lambda y: fechaValida(y))

mes_metlife = pd.DataFrame()
mes_metlife = pd.concat(
    [
        subir_web,
        subir_sql,
        subir_correo,
        
    ],
    axis=0,
)
mes_metlife["fecha_rut"] = pd.to_datetime(fecha, yearfirst=True)

mes_metlife = mes_metlife.drop_duplicates(
    [
        "rut_rut","poliza_rut",
    ],
    keep="last",
)
mes_metlife.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_22780\846486643.py:21: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  subir_sql["hasta_rut"] = metlife_sftp["hasta"].apply(lambda y: fechaValida(y))


,poliza_rut,rut_rut,nombre_rut,mes_rut,año_rut,Llave,desde_rut,hasta_rut,fecha_rut
82,340022579,118953509,Pablo Muñoz Valencia,7,2023,34002257911895350972023,2024-03-15 02:31:36,2024-03-15 02:31:36,2023-07-01
98,340023322,9082855K,Amable Ricardo Salinas Ramirez,7,2023,3400233229082855K72023,2024-03-15 02:31:36,2024-03-15 02:31:36,2023-07-01
371,340023905,116360195,Claudio Ernesto Aguirre Gonzalez,7,2023,34002390511636019572023,2024-03-15 02:31:36,2024-03-15 02:31:36,2023-07-01
382,340017375,62796138,Hugo Eduardo Cubillos Alarcon,7,2023,3400173756279613872023,2024-03-15 02:31:36,2024-03-15 02:31:36,2023-07-01
830,340007352,170353595,Pamela Alejandr Asencio Garcia,7,2023,34000735217035359572023,2024-03-15 02:31:36,2024-03-15 02:31:36,2023-07-01


NUMERO RUTS POR POLIZA

In [ ]:
mes_metlife.shape
print ( str(mes) + "-" + str(año) + " " + str(mes_metlife.shape[0]))

7-2023 403641


NUMERO RUT UNICOS

In [ ]:
mes_metlife2 = mes_metlife.drop_duplicates(
    [
        "rut_rut"
    ],
    keep="last",
)
mes_metlife2.shape
print (str(mes) + "-" + str(año) + " " + str(mes_metlife2.shape[0]))

7-2023 309889


In [ ]:
#subir_web["Llave"]
mes_metlife["esta"] =  mes_metlife["Llave"].isin(rut_unico_tiempo_["Llave"])
mes_metlife = mes_metlife[(mes_metlife["esta"] == False)]

mes_metlife = mes_metlife[
    [
        "poliza_rut",
        "rut_rut",
        "nombre_rut",
        "mes_rut",
        "año_rut",
        "Llave",
        "desde_rut",
        "hasta_rut",
    ]
]
mes_metlife.shape

(403641, 8)

In [ ]:
mes_metlife.to_sql(
    name="rut_unico_tiempo",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)

403641

In [ ]:
subir_unicos = pd.DataFrame()
subir_unicos["Rut_T"] = mes_metlife2["rut_rut"]
subir_unicos["Rut_C"] = mes_metlife2["rut_rut"]
subir_unicos["Nombre"] = mes_metlife2["nombre_rut"]
subir_unicos["Llave"] = mes_metlife2["rut_rut"] +  mes_metlife2["rut_rut"]


subir_unicos.shape

(309889, 4)

In [ ]:
subir_unicos["esta"] =  subir_unicos["Rut_T"].isin(rut_unicos["Rut_T"])
subir_unicos = subir_unicos[(subir_unicos["esta"] == False)]

subir_unicos = subir_unicos[
    [
        "Rut_T",
        "Rut_C",
        "Nombre",
        "Llave",
    ]
]
subir_unicos.shape
subir_unicos.to_sql(
    name="rut_unicos",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)

12161